In [ ]:
# ! pip install git+https://github.com/keunwoochoi/kapre.git

In [1]:
import IPython
import IPython.display as ipd
import librosa
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
from perceiver import Perceiver
from tensorflow import keras
from tqdm import tqdm

from lib.float2d_to_rgb_layer import Float2DToRGB
from lib.melspectrogram_layer import MelSpectrogram
from lib.power_to_db_layer import PowerToDb
from lib.utils import float2d_to_rgb, save_keras_model
from src.config import c
from src.generator import Generator
from src.services import get_msg_provider, get_wave_provider

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, GlobalAveragePooling2D, Dense, Softmax
from kapre import STFT, Magnitude, MagnitudeToDecibel
from kapre.composed import get_melspectrogram_layer, get_log_frequency_spectrogram_layer

In [3]:
wave_p = get_wave_provider(c)

df = pd.read_pickle("/app/_work/dataset-C.pickle")

g = Generator(
    df=df,
    wave_provider=wave_p,
    batch_size=1,
    rating_as_sw=False,
    rareness_as_sw=False,
)

In [4]:
# 6 channels (!), maybe 1-sec audio signal, for an example.
input_shape = (160000, 1)
sr = 32000

In [14]:
N = 32000

In [15]:
waves = np.zeros((N, 160000), dtype=np.float16)
ys = np.zeros((N, 319), dtype=np.float16)

In [16]:
for i in tqdm(range(N)):
    x, y, sw = g.__getitem__(i)
    waves[i] = x["i_wave"][0]
    ys[i] = y[0]

100%|██████████| 32000/32000 [02:28<00:00, 215.71it/s]


In [10]:
waves = waves[..., np.newaxis]

In [11]:
waves.shape, ys.shape

((1024, 160000, 1, 1), (1024, 319))

In [18]:
get_melspectrogram_layer?

Signature:
get_melspectrogram_layer(
    input_shape=None,
    n_fft=2048,
    win_length=None,
    hop_length=None,
    window_name=None,
    pad_begin=False,
    pad_end=False,
    sample_rate=22050,
    n_mels=128,
    mel_f_min=0.0,
    mel_f_max=None,
    mel_htk=False,
    mel_norm='slaney',
    return_decibel=False,
    db_amin=1e-05,
    db_ref_value=1.0,
    db_dynamic_range=80.0,
    input_data_format='default',
    output_data_format='default',
    name='melspectrogram',
)
Docstring:
A function that returns a melspectrogram layer, which is a `keras.Sequential` model consists of
`STFT`, `Magnitude`, `ApplyFilterbank(_mel_filterbank)`, and optionally `MagnitudeToDecibel`.

Args:
    input_shape (None or tuple of integers): input shape of the model. Necessary only if this melspectrogram layer is
        is the first layer of your model (see `keras.model.Sequential()` for more details)
    n_fft (int): number of FFT points in `STFT`
    win_length (int): window length of `STFT`


In [17]:
model = Sequential()
# A STFT layer
model.add(STFT(n_fft=2048, win_length=2018, hop_length=1024,
               window_name=None, pad_end=False,
               input_data_format='channels_last', output_data_format='channels_last',
               input_shape=input_shape))
model.add(Magnitude())
model.add(MagnitudeToDecibel())  # these three layers can be replaced with get_stft_magnitude_layer()
# Alternatively, you may want to use a melspectrogram layer

melgram_layer = get_melspectrogram_layer()

# or log-frequency layer
# log_stft_layer = get_log_frequency_spectrogram_layer() 

# add more layers as you want
model.add(Conv2D(32, (3, 3), strides=(2, 2)))
model.add(BatchNormalization())
model.add(ReLU())
model.add(GlobalAveragePooling2D())
model.add(Dense(ys.shape[1]))
model.add(Softmax())

model.compile(
    optimizer='binary_crossentropy',
    loss="bce",
    metrics=[
        tfa.metrics.F1Score(
            num_classes=ys.shape[1],
            threshold=0.5,
            average="micro",
        ),
    ],
)

# Compile the model
# model.compile('adam', 'categorical_crossentropy') # if single-label classification

# train it with raw audio sample inputs
# for example, you may have functions that load your data as below.
# x = load_x() # e.g., x.shape = (10000, 6, 44100)
# y = load_y() # e.g., y.shape = (10000, 10) if it's 10-class classification
# then..
model.fit(x=waves, y=ys, batch_size=8, verbose=1, validation_split=0.2, epochs=10)
# Done!

Epoch 1/10
3200/3200 [==============================] - 35s 11ms/step - loss: 0.0248 - f1_score: 0.0000e+00 - val_loss: 0.0244 - val_f1_score: 0.0000e+00
Epoch 2/10
1299/3200 [===========>..................] - ETA: 13s - loss: 0.0244 - f1_score: 0.0000e+00

KeyboardInterrupt: 